<a href="https://colab.research.google.com/github/minjun1/realnvp_seismic/blob/main/notebook_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1 - Normal Moveout

In [ ]:
!python -m pip install "sep-plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@fe0395fd514f6aac60b1c696c0ee99ca42c3ad89"

## Abstract

We will look at the normal moveout (NMO) operator and examine its implementation. You will code the forward operator with linear interpolation to create the dataset. You will then code the adjoint operator and run the dot product test. Finally, you will run two least squares optimizations using two different steppers and analyze the results.

## Normal moveout operator

The NMO operator is an example of a summation transform operator. This operator approximates the reflection travel time given the medium velocity above a reflector in a 1D Earth. In this operator the model space axes are intercept time (analogous to traveltime depth) and slowness, whereas the data space axes are time and offset. The mapping equation can be written as:
\begin{equation}\label{eq:nmo}
  t = \sqrt{s^2 x^2 + \tau^2},
\end{equation}
where $t$ is time, $s$ is slowness, $x$ is offset and $\tau$ is intercept time. The operator loops over $s$, $x$, $\tau$, and computes $t$. Note that the computed time often falls between the samples of the time axis. Therefore, interpolating the values can increase the accuracy of the operator.

This operator is your most basic imaging operator (it is also known as a beamformer or hyperbolic radon transform). For this lab, we will see that the application of the forward of this operator maps spikes defined in the $s$ and $\tau$ domain to hyperbolas in the $t$ and $x$ domain.

One final thing to note is that for all plots below we have replaced $\tau$ with $z$ on the vertical axis. So instead of seeing a $\tau$-$s$ space, you will see a $z$-$s$ space.

## Your assignment

In this lab you will be solving your first inverse problem. You will first write the code for the forward and the adjoint of the NMO operator as described above. Then, you will apply both the forward and the adjoint operator and examine their outputs. Additionally, you will need to make sure that your operator passes the dot product test. 

Finally, you will run two inversions with different inversion algorithms. One you will run with the steepest descent algorithm and the other with a conjugate gradient algorithm.

### Code skeleton

In the cell below, we have provided the majority of the code necessary for completing the implementation of the NMO operator. As described above, your main task will be to complete the two functions at the bottom of this cell that implement the forward and adjoint of this operator.

In [ ]:
from genericSolver.pyOperator import Operator as operator
from genericSolver.pyVector  import vector
from numba import jit, int32, float32
import numpy as np

class slow(operator):
    """
    2D slowness operator that maps spikes in slowness and depth
    to hyperbolas in time and space
    """

    def __init__(self, dom, rng):
        """
        Initialize operator and saves the space of the operator
        """
        if not isinstance(dom,vector):
          raise Exception("Expecting domain to be a python vector")

        if not isinstance(rng,vector):
          raise Exception("Expecting range to be a python vector")

        # Store the vector space of the domain and range
        super().__init__(dom,rng)
        
        # Get model axes
        zaxis = dom.get_hyper().get_axis(1)
        qaxis = dom.get_hyper().get_axis(2)
        # Get data axes
        taxis = rng.get_hyper().get_axis(1)
        xaxis = rng.get_hyper().get_axis(2)

        # Get model dimensions
        self._oq = qaxis.o; self._dq = qaxis.d
        self._oz = zaxis.o; self._dz = zaxis.d
        # Get data dimensions
        self._ox = xaxis.o; self._dx = xaxis.d
        self._ot = taxis.o; self._dt = taxis.d

    def forward(self,add,modl,data):
        """
        Applies the forward operator:
        Spikes in depth-slowness to hyperbolas in time-space

        Parameters:
          add - boolean whether or not add to the data vector or zero it first
          modl - slowness model (s,z)
          data - hyperbolas (t,x)
        """
        print(type(modl),type(self.domain),"CHECK")
        self.checkDomainRange(modl,data)

        # Zero the data if add == false
        if not add:
            data.zero()

        forward2D_1(self._oq,self._dq,
                    self._oz,self._dz,
                    self._ox,self._dx,
                    self._ot,self._dt,
                    modl.get_nd_array(),data.get_nd_array())

    def adjoint(self,add,modl,data):
        """
        Applies the adjoint operator:
        Hyperbolas in time-space to spikes in depth-slowness

        Parameters:
          add - boolean whether or not to add the model vector or zero it first
          modl - slowness model (s,z)
          data - hyperbolas (t,x)
        """

        if not add:
            modl.zero()

        adjoint2D_1(self._oq,self._dq,
                    self._oz,self._dz,
                    self._ox,self._dx,
                    self._ot,self._dt,
                    modl.get_nd_array(),data.get_nd_array())

####### Please complete the two functions below #######
@jit(nopython=True)
def forward2D_1(oq,dq,oz,dz,ox,dx,ot,dt,modl,data):
    #TODO: implement forward operator
    pass

@jit(nopython=True)
def adjoint2D_1(oq,dq,oz,dz,ox,dx,ot,dt,modl,data):
    #TODO: implement adjoint operator
    pass

### Testing the forward operator

With your code now complete, lets first test the forward operator. We can do this by looking at the application of our operator to some impulses. The following cell creates four impulses that indicate events of different slowness that originated at different depths.

In [ ]:
from sep_plot import Grey
from sep_python.hypercube import Hypercube, Axis
import sep_python.modes    #Import SEP python module
io=sep_python.modes.default_io  #Get default IO that expects SEPlib datasets and uses sepVectors

#Create model space
modt=io.get_reg_vector(Hypercube.set_with_ns(ns=[101,101], ds=[0.04,0.004],os=[0.0,0.2]))
modv = modt.get_nd_array()

modv[75][20]=1.
modv[60][40]=1
modv[50][60]=1
modv[40][80]=1

In [ ]:
import holoviews as hv 
hv.extension('bokeh','matplotlib')

grey=Grey(modt,label1="z",label2="s",bclip=-0.5,eclip=0.5,height=700,width=500,invert_yaxis=True,colorbar=True)
grey.image()

In the following cell we will apply the forward to the input impulses to see if it produces hyperbolas:

In [ ]:
# Create data
dat = io.get_reg_vector(Hypercube.set_with_ns(ns=[101,101], ds=[0.04,0.1], os=[0.0,-5]))
op = slow(modt,dat)

# Apply the forward
op.forward(False,modt,dat)

hv.extension('bokeh','matplotlib')
grey=Grey(dat,label1="t",label2="x",height=700,width=500,invert_yaxis=True,colorbar=True)
grey.image()

If you see four hyperbolas, good chances are you have written code for the forward operator correctly. 

### Testing the adjoint operator
Next you need to check if you have the correct adjoint of the forward operator. We will do this by running the dot product test. If running the following cell does not cause an error then you have correctly implemented the adjoint of your forward operator.

In [ ]:
op.dotTest(verbose=True)

Now let's examine the output of the adjoint when applied to the modeled data from the application of the forward. First we clone the space of the spike model to create our output image

In [ ]:
adj = modt.clone()
adj.zero()

op.adjoint(False,adj,dat)

grey=Grey(adj,label1="z",label2="s",height=700,width=500,invert_yaxis=True,colorbar=True)
grey.image()

 Take some time to examine the output and answer the following questions:
 1. **Is this adjoint a good approximation to the inverse?**
 2. **Although the four spikes were similar in the true model, they are different in the adjoint results. What are the differences? What caused those differences?**
 3. **In a real situation in which we do not have knowledge of the true model, we can only measure the accuracy of the results in data space. Apply the forward operator on the adjoint model and plot the reconstructed data. How does this data compare to the observed data (be elaborate)?**

### Steepest descent and conjugate gradient inversions

In this last section of the lab, you will now attempt to compute the least-squares inverse of the NMO operator. We will do so iteratively using both the steepest-descent and conjugate gradient iterative algorithms. We will then compare the outputs of each.

#### Steepest descent
The following cell has all of the code necessary for you to run a steepest descent inversion. Just as a review/outline of the code, it works as follows:

  1. Object creation. The code first creates three objects necessary for running the inversion. 
       1. *Problem object*. This is the main class for defining the inversion problem we desire to solve. Among other things, this object knows how to compute our objective function and its gradient which are necessary for the gradient-based inversion.
       2. *Stopper object*. This object determines the stopping criteria which will be used for terminating the inversion. In this case we desire to run the inversion for only 20 iterations
       3. *Solver object*. This object use both the problem and the stopper to run the gradient-based inversion.
     
  2. Next we run the inversion by passing the problem object to the solver object which uses the defined gradient functions and objective function evaluation functions to perform the iterative optimization.
  
  3. Finally, we get the results from the problem object as it contains all of the  inversion variables

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear as problem 
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper as stopper
# Create inversion objects
modt.zero()
probsd = problem(modt,dat,op)
niterStopper = stopper(niter=20)
sd = LCGsolver(niterStopper,steepest=True)

# Run the inversion
sd.run(probsd,verbose=True)
sdOut=probsd.model

grey=Grey(sdOut,label1="z",label2="s",bclip=-0.09,eclip=0.09,height=700,width=500,invert_yaxis=True,colorbar=True)
grey.image()

#### Conjugate gradient
Finally, we run the same inversion problem but with a conjugate gradient solver

In [ ]:
modt.zero()
probcg = problem(modt,dat,op)
niterStopper = stopper(niter=20)
cg = LCGsolver(niterStopper,steepest=False)

cg.run(probcg,verbose=True)
cgOut = probcg.model

grey=Grey(cgOut,label1="z",label2="s",bclip=-0.09,eclip=0.09,height=700,width=500,invert_yaxis=True,colorbar=True)
grey.image()

**Compute and plot the residual of each optimization by subtracting the reconstructed data from the observed data. Without considering their power, which residual indicates better convergence? Why?**
